# Relatório de desenvolvimento do módulo `doc_data` versão `0.5.7`
**Autor**: Caio Geraldes

## Introdução

O módulo `doc_data`, escrito em `Python`, tem como função auxiliar a coleta dos dados necessários para a investigação proposta em meu doutorado.
O módulo se aproveita do corpus anotado `Diorisis` e do pacote de processamento de linguagem natural `stanza` para produzir árvores de dependência que filtram sentenças em que seria plausível encontrar atração de caso de predicados secundários de orações infinitivas do grego antigo.
Os detalhes de funcionamento do módulo está especificado no repositório no GitHub [caiogeraldes/doc_data](www.github.com/caiogeraldes/doc_data) e nos submódulos, funções e classes que compõem o projeto.

# Metodologia

O módulo `doc_data` segue, na versão `0.5.7`, as seguintes etapas para filtrar os dados:

1. Os textos disponibilizados no corpus `Diorisis` são separados em um ficheiro próprio;
2. Os textos são processados para retirar apenas os tokens de texto e metadata (autoria, documento etc);
3. Os tokens são reprocessados pelo modelo treinado com o corpus `Perseus` com ajuda do pacote `stanza`, produzindo sentenças e seus constituintes tokenizados com relações de dependência, análise morfológica e indexação;
4. Os tokens são armazenados em um banco de dados `NoSQL`, neste caso um `MongoDB`, para permitir filtragens mais precisas;
5. Filtram-se os tokens:
    - que pertençam a sentenças contendo um verbo principal de interesse (`mvi` = *main verb of interest*), selecionados em [mvi.csv](https://github.com/caiogeraldes/doc_data/blob/main/data/mvi.csv);
    - que pertençam a sentenças contendo um infinitivo dependente do `mvi`;
    - que pertençam a sentenças contendo um dativo ou genitivo dependente do `mvi`.
    
## Efeitos da filtragem

O código abaixo permite inspecionar o resultado da filtragem.

In [13]:
from doc_data.main import MONGO
from doc_data.db import mongo
from doc_data.tools import gen_sent, gen_meta, gen_hits
db = mongo(MONGO)
tokens = db.tokens
interest_tokens = db.interest_tokens
print(f"Sentenças totais:\t\t{len(tokens.distinct('text-sentence'))}")
print(f"Tokens totais:\t\t\t{tokens.count_documents({})}")
print(f"Sentenças após filtragem:\t{len(interest_tokens.distinct('text-sentence'))}")
print(f"Tokens após filtragem:\t\t{interest_tokens.count_documents({})}")

Connected successfully
Sentenças totais:		133801
Tokens totais:			2418964
Sentenças após filtragem:	2807
Tokens após filtragem:		86016


O corpus inicial `tokens` contém $92.551$ sentenças contendo $2.855.303$ tokens.
Após os três passos de filtragem, o corpus final `interest_tokens`, contém $3.621$ sentenças contendo $199.656$ tokens, $3.9\%$ do número de sentenças original ($7\%$ do número de tokens original).

## Qualidade da filtragem

**PÉSSIMA**

Seleciono aleatoriamente abaixo 36 sentenças ($\simeq 1\%$ do total) filtradas para averiguar se o método de filtragem produziu dados utilizáveis (a `seed` 36 foi escolhida arbitrariamente para que sempre se produzam as mesmas frases):

In [14]:
import random
random.seed(36)
xobj = db.xobjquery
tss = xobj.distinct("text-sentence")
for ts in random.choices(tss, k=36):
    print(gen_meta(interest_tokens, ts))
    print(gen_hits(interest_tokens, ts, [db.mviquery, db.infquery, xobj]))
    print(gen_sent(interest_tokens, ts))
    print()

Plato (0059) - Laws (034)
{'mviquery': ['δοκεῖς'], 'infquery': ['ἐστοχάσθαι'], 'xobjquery': ['μοι']}
ὀρθότατά γε , ὦ ξένε , λέγεις , καὶ καθάπερ μάντις , ἀπὼν τῆς τότε διανοίας τοῦ τιθέντος αὐτά , νῦν ἐπιεικῶς μοι δοκεῖς ἐστοχάσθαι καὶ σφόδρα ἀληθῆ λέγειν .

Demosthenes (0014) - On the Peace (005)
{'mviquery': ['ὑπάρξειν'], 'infquery': ['ἀποδοθήσεσθαι'], 'xobjquery': ['αμφιπόλεως']}
τὸ τρίτον δʼ , ὦ ἄνδρες *̓ αθηναῖοι , ̔ καὶ μόνον ἓν τοῦτʼ εἰπὼν ἔτι καὶ δὴ περὶ ὧν παρελήλυθʼ ἐρῶ ̓ , ἡνίκα τοὺς ὅρκους τοὺς περὶ τῆς εἰρήνης ἀπειληφότες ἥκομεν οἱ πρέσβεις , τότε Θεσπιάς τινων καὶ Πλαταιὰς ὑπισχνουμένων οἰκισθήσεσθαι , καὶ τοὺς μὲν Φωκέας τὸν Φίλιππον , ἂν γένηται κύριος , σώσειν , τὴν δὲ Θηβαίων πόλιν διοικιεῖν , καὶ τὸν *̓ ωρωπὸν ὑμῖν ὑπάρξειν , καὶ τὴν Εὔβοιαν ἀντʼ *̓ αμφιπόλεως ἀποδοθήσεσθαι , καὶ τοιαύτας ἐλπίδας καὶ φενακισμούς , οἷς ὑπαχθέντες ὑμεῖς οὔτε συμφόρως οὔτʼ ἴσως καλῶς προεῖσθε Φωκέας , οὐδὲν τούτων οὔτʼ ἐξαπατήσας οὔτε σιγήσας ἐγὼ φανήσομαι , ἀλλὰ προειπὼν ὑμῖν , ὡς οἶδʼ

{'mviquery': ['πρέπει'], 'infquery': ['μαλακισθῆναί'], 'xobjquery': ['ὧν']}
ἀνθʼ ὧν μὴ μαλακισθῆναί τινα πρέπει μηδὲ τὸ ἀκινδύνως ἀπελθεῖν αὐτοὺς κέρδος νομίσαι .

Sophocles (0011) - Ajax (003)
{'mviquery': ['δοκεῖν'], 'infquery': ['φρονεῖν'], 'xobjquery': ['ἐμοὶ']}
σοὶ μὲν δοκεῖν ταῦτʼ ἔστʼ , ἐμοὶ δʼ ἄγαν φρονεῖν .

Plato (0059) - Laws (034)
{'mviquery': ['πρέποντα', 'πιστεύει', 'πρέπον'], 'infquery': ['ἐξηυρηκέναι'], 'xobjquery': ['γένεσιν']}
ὁπότε τις οὖν καὶ ὁπηνίκα τῶν πέντε καὶ εἴκοσι γεγονότων ἔτη , σκοπῶν καὶ σκοπούμενος ὑπʼ ἄλλων , κατὰ νοῦν ἑαυτῷ καὶ πρέποντα εἰς παίδων κοινωνίαν καὶ γένεσιν ἐξηυρηκέναι πιστεύει , γαμείτω μὲν πᾶς ἐντὸς τῶν πέντε καὶ τριάκοντα ἐτῶν , τὸ δὲ πρέπον καὶ τὸ ἁρμόττον ὡς χρὴ ζητεῖν , πρῶτον ἐπακουσάτω ·

Xenophon (0032) - Cyropaedia (007)
{'mviquery': ['ἔξεστι'], 'infquery': ['κτήσασθαι'], 'xobjquery': ['ἡμῶν']}
φιλίαν δὲ παρὰ τίνων ἄν ποτε λάβοις τοσαύτην ὅσην σοι παρʼ ἡμῶν ἔξεστι κτήσασθαι νῦν ;

Xenophon (0032) - Anabasis (006)
{'mviquery': ['πισ